# Lecture 3.1 - Functions and State

In [8]:
class BankAccount {
    private var balance = 0
    def deposit(amount: Int): Unit = {
        if (amount > 0) balance = balance + amount
    }
    def withdraw(amount: Int): Unit = {
        if (balance >= amount && amount >= 0) {
            balance = balance - amount
            balance
        }
        else throw new Error("INSUFFICIENT FUNDS BRO")
    }
}

defined class BankAccount

In [13]:
val acct = new BankAccount

acct: BankAccount = ammonite.$sess.cmd7$Helper$BankAccount@6473f2aa

In [14]:
acct.deposit(50)
acct.withdraw(20)
acct.withdraw(20)
acct.withdraw(15)

: 

# Lecture 3.2 - Identity and Change

Assignment poses the problem of deciding whether two objects are the same.
When one excludes assigments and one writes
```scala
val x = E
val y = E
```
where `E` is an arbitrary expression, then it is reasonable to assume that `x` and `y` are the same.  
This property is usually called **referential transparency**.

The precise meaning of "being the same" is defined by the property of **operational equivalence**.

Suppose we have two definitions `x` and `y`. Then they are operationally equivalent if no possible test can distinguish between them.

# Lecture 3.3 - Loops

# Lecture 3.4 - Discrete Event Simulation

A wire is a connected component until it hits a gate.
```scala
val a, b, c = new Wire
```

Gates (each has a side effect that creates a gate)
```scala
def inverter(input: Wire, output: Wire): Unit
def andGate(a1: Wire, a2: Wire, output: Wire): Unit
def orGate(b2: Wire, b2: Wire, output: Wire): Unit
```

### Half-adder
<img src="half_adder.png"/>
```scala
def halfAdder(a: Wire, b: Wire, s: Wire, c: Wire): Unit = {
    val d = new Wire
    val e = new Wire
    orGate(a, b, d)
    andGate(a, b, c)
    inverter(c, e)
    andGate(d, e, s)
}
```

### Full-adder
<img src="full_adder.png"/>
```scala
def fullAdder(a: Wire, b: Wire, cin: Wire, sum: Wire, cout: Wire): Unit = {
    val s = new Wire
    val c1 = new Wire
    val c2 = new Wire
    halfAdder(b, cin, s, c1)
    halfAdder(a, s, sum, c2)
    orGate(c1, c2, cout)
```

# Lecture 3.5 - Discrete Event Simulation: API and Usage

### Simulation signature
```scala
trait Simulation {
    def currentTime: Int = ???
    def afterDelay(delay: Int)(block => Unit): Unit = ???
    def run(): Unit = ???
```

### Class diagram
1. Simulation  
2. Gates (Wire, AND, OR, INV)
3. Circuits (HA, Adder)
4. My Simulation

Wire class API
```scala
getSignal: Boolean
setSignal(sig: Boolean): Unit
addAction(a: Action): Unit
```

Implementation
```scala
class Wire {
    private var sigVal = false
    private var actions: List[Action] = List()
    def getSignal: Boolean = sigVal
    def setSignal(s: Boolean): Unit = {
        if (s != sigVal) {
            sigVal = s
            actions foreach (_())
        }
    }
    def addAction(a: Action): Unit = {
        actions = a :: actions
        a()
    }
}
```
            


Inverter implementation

```scala
def inverter(input: Wire, output: Wire): Unit = {
    def inverterAction(): Unit = {
        val inputSig = input.getSignal
        afterDelay(InverterDelay) { output setSignal !inputSig }
    }
    input addAction inverterAction
}
```

AND and OR implementations

# Lecture 3.6

### Simulation

```scala
trait Simulation {
    type Action = () => Unit
    case class Event(time: Int, action: Action)
    private type Agenda = List[Event]
    private var agenda: Agenda = List()
} 
```

```scala
private var curtime = 0
def curtime: Int = curtime
```

```scala
def afterDelay(delay: Int)(block: => Unit): Unit = {
    val item = Event(curtime + delay, () => block)
    agenda = insert(agenda, item)
}
```

```scala
private def insert(ag: List[Event], item: Event): List[Event] = ag match {
    case first :: rest if first.time <= item.time =>
        first :: insert(rest, item)
    case _ => 
        item :: ag
}
```

### Event handling loop
```scala
private def loop(): Unit = agenda match {
    case first :: rest =>
        agenda = rest
        curtime = first.time
        first.action()
        loop()
    case Nil =>
}
```

### Probes
```scala
def probe(name: String, wire: Wire): Unit = {
    def probeAction(): Unit = {
        println(s"$name $currentTime $value = ${wire.getSignal}")
    }
    wire addAction probeAction
}
```

### Define Technology Related Parameters

```scala
trait Parameter {
    def InverterDelay = 2
    def AndGateDelay = 3
    def OrGateDelay
}

obj sim extends Circuits with Parameters
```

## Concrete Simulation